In [75]:
%matplotlib inline

In [242]:
import pandas as pd
import subprocess
import re
import json

In [260]:
def get_dates(year):
    events = ['Tu BiShvat',
          'Erev Purim',
          'Erev Pesach',
          'Erev Shavuot',
          'Erev Tish\'a B\'Av',
          'Erev Rosh Hashana',
#           'Erev Yom Kippur',
#           'Erev Sukkot',
          'Chanukah: 1 Candle']
    hebcal_str = subprocess.run('../hebcal-master/./hebcal ' + str(year), shell=True, stdout=subprocess.PIPE, encoding='utf8').stdout
    results = []
    is_leap = 'Adar II' in hebcal_str
    for event in events:
        date_search = re.search(r'(([0-9]{1,2})/([0-9]{1,2})/[0-9]{4}) ' + event, hebcal_str)
        date = date_search.group(1)
        month = int(date_search.group(2))
        day = int(date_search.group(3))
        results.append({'event': event, 'year': year, 'month': month, 'day': day, 'leap': is_leap})
    return results

In [261]:
dates = []
for y in range(1800, 2200):
    dates.extend(get_dates(y))

In [263]:
# export raw data
with open('data.json', 'w') as file:
    json.dump(dates, file)

In [268]:
# export aggregated data
pd.DataFrame(pd.DataFrame(dates).groupby(['event', 'month', 'day']).size(), columns=['count']).reset_index().to_json('agg_data.json', orient='records')

event    Tu BiShvat
month            12
day              31
count            17
dtype: object